In [1]:
from dotenv import load_dotenv
import os
# Load environment variables from the .env file
load_dotenv()

True

Extract

In [2]:

# Import the required function for downloading from S3
from download import download_from_s3
# Download files from the specified S3 bucket
# We used n_documents=1000 but it takes a lot of time
# For demo purposes we set it on 5, and it loads 3 PDFs
download_from_s3(n_documents=5)

Downloaded: nasdaq_annual_reports/.DS_Store -> dataset/.DS_Store
Downloaded: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2019.pdf -> dataset/NASDAQ_FLWS_2019.pdf
Downloaded: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2020.pdf -> dataset/NASDAQ_FLWS_2020.pdf
Downloaded: nasdaq_annual_reports/1-800-flowerscom/NASDAQ_FLWS_2021.pdf -> dataset/NASDAQ_FLWS_2021.pdf


Transform

In [3]:
import os
# Import the required functions for splitting the downloaded dataset
from data_split import process_and_save_dataset
# Define the folder path containing the PDF files
pdf_folder_path = os.environ['LOCAL_DIRECTORY']
# Process the PDF files and save the resulting DataFrame to a CSV file
df = process_and_save_dataset(pdf_folder_path, file_name=os.environ['DATASET_NAME'])

/home/gonza/miniconda3/envs/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/gonza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gonza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.1 MB/s eta 0:00:00m eta 0:00:010:0101
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Preprocessing: 100%|██████████| 1/1 [00:00<00:00,  8.46docs/s]


Load splits to the vector database


In [4]:
file_path = os.path.join(os.environ['LOCAL_DIRECTORY'], os.environ['DATASET_NAME'])
file_path

'dataset/dataset.csv'

In [5]:
# Import the required function for processing the documents
from load import process_documents
# Process the documents using the specified function
process_documents()

/home/gonza/miniconda3/envs/env/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Try if it worked

In [6]:
# Import chroma
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1")
persist_directory=os.environ['PERSIST_DIRECTORY']


# Load the database from existing embeddings
db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings,
    collection_name=os.environ['COLLECTION'])

print('This Vectorstore has this many chunks:')
print(len(db.get()['ids']))
print("")
print('And its composed by:')
print(db.get().keys())


This Vectorstore has this many chunks:
117

And its composed by:
dict_keys(['ids', 'embeddings', 'metadatas', 'documents'])
